# Segmentator #

In [20]:
import os
import numpy as np
from PIL import Image
import PIL
PIL.Image.MAX_IMAGE_PIXELS = None

In [23]:
def split_tiff_to_tiles(input_folder, output_dir, patch_size=1024, overlap=0.1):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.tiff'):
            input_path = os.path.join(input_folder, file_name)
            
            image = Image.open(input_path)
            image_array = np.array(image)

            output_folder = os.path.join(output_dir, os.path.splitext(file_name)[0])
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            step = int(patch_size * (1 - overlap))

            height, width = image_array.shape[:2]
            for y in range(0, height - patch_size + 1, step):
                for x in range(0, width - patch_size + 1, step):
                    tile = image_array[y:y+patch_size, x:x+patch_size]

                    if not np.any(tile):
                        continue  

                    total_pixels = patch_size * patch_size

                    if tile.ndim == 3:
                        white_pixels = np.sum(np.all(tile == [255, 255, 255], axis=-1))
                        black_pixels = np.sum(np.all(tile == [0, 0, 0], axis=-1))
                    else:  
                        white_pixels = np.sum(tile == 255)
                        black_pixels = np.sum(tile == 0)

                    if (white_pixels + black_pixels) / total_pixels >= 0.5:
                        continue  

                    tile_image = Image.fromarray(tile)
                    tile_output_path = os.path.join(output_folder, f'tile_{y}_{x}.tiff')
                    tile_image.save(tile_output_path)

            print(f"Saves tiles to: {file_name}")


In [27]:
split_tiff_to_tiles(
    input_folder='../preprocessedData/tissue_regions/1M01',
    output_dir='../preprocessedData/tiles/1M01',
    patch_size=1280,  
    overlap=0.75
)

KeyboardInterrupt: 